# Univ.AI Hackathon 

## Hello and Welcome to the start of Ayan's Attempt to Geofferey Hinton's Fellowship 

## I am honored to participate in such an event and present my work. 

## Let's begin with the notebook then. 

# Contents 

## 1. [Libraries](#libraries)
## 2. [Data Visualization](#data_visualization)
## 3. [Feature Engineering](#feature_engineering)
## 4. [Data Modelling](#data_modelling)
## 5. [Output](#output)

# Libraries <a id='libraries'></a>
Let's start with all the imports and get done with the boring part 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import re
from category_encoders import LeaveOneOutEncoder
from sklearn.preprocessing import OrdinalEncoder # Not Needed Right now
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import optuna
import lightgbm
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
train=pd.read_csv('/kaggle/input/ghf1-hackathon/Training Data.csv')
test=pd.read_csv('/kaggle/input/ghf1-hackathon/Test Data.csv')
sub=pd.read_csv('/kaggle/input/ghf1-hackathon/Sample Prediction Dataset.csv')

# Data Visualization <a id='data_visualization'></a>
### Let's look with what data we will be working with in this Problem 

In [ ]:
train.head()

In [ ]:
train.shape

### So the data has 2.5 lakh entires, that's a lot of credit card data

### Let's see what is the distribution of the target column in this data 

In [ ]:
train['risk_flag'].value_counts()/len(train)*100

### Looks like the data is highly skewed. 

### Check for null values in train and test set 

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

### So no null values, the next step is to go for getting info from train set

### As see if we can find anything solid 

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.nunique()

### Id column looks a bit redundant here, dropping it will be the best thing to do 

In [ ]:
train.drop(columns=['Id'],inplace=True)
test.drop(columns=['id'],inplace=True) # Note:- Keep in mind the 'id' for test is different, i is lower-case in this case, so case in point :-)

In [ ]:
total_columns=['age', 'experience', 'married', 'house_ownership','car_ownership', 'profession', 'city',
               'state', 'current_job_years','current_house_years',]

for i in total_columns:
    print(f'Column :{i} New Elements :{set(test[i].unique())-set(train[i].unique())}')
    print(f'Number of New Elements {len(set(test[i].unique())-set(train[i].unique()))}')
    print('')

### This is interesting, we can see that we have a lot of entries with '-' , 'upper case' , '_' or anything else creating a new entry

### Before we proceed any further, let's fix this error

In [ ]:
def convert_profession(x):
    prof = x
    prof = re.sub(r'\-',' ',prof)
    prof = re.sub(r'\_',' ',prof)
    prof = re.sub(r'\[[0-9]\]',' ',prof)
    prof = prof.split()
    prof = ' '.join(prof)
    
    prof = prof.lower()
    return prof

def convert_state(x):
    state = x
    state = re.sub(r'\-',' ',state)
    state = re.sub(r'\_',' ',state)
    state = re.sub(r'\[[0-9]\]',' ',state)
    state = state.split()
    state = ' '.join(state)
    
    state = state.lower()
    return state

def convert_city(x):
    city = x
    city = re.sub(r'\-',' ',city)
    city = re.sub(r'\_',' ',city)
    city = re.sub(r'\[[0-9][0-9]*\]',' ',city)
    city = city.split()
    city = ' '.join(city)
    
    city = city.lower()
    return city

train['profession'] = train['profession'].apply(convert_profession)
test['profession'] = test['profession'].apply(convert_profession)

train['state'] = train['state'].apply(convert_state)
test['state'] = test['state'].apply(convert_state)

train['city'] = train['city'].apply(convert_city)
test['city'] = test['city'].apply(convert_city)

### Confirm it for surety, sice being extra carefull is never bad 

In [ ]:
total_columns=['age', 'experience', 'married', 'house_ownership','car_ownership', 'profession', 'city',
               'state', 'current_job_years','current_house_years',]

for i in total_columns:
    print(f'Column :{i} New Elements :{set(test[i].unique())-set(train[i].unique())}')
    print(f'Number of New Elements {len(set(test[i].unique())-set(train[i].unique()))}')
    print('')

### Done and Done

### Moving on now, some basic visualisations for inferences 

In [ ]:
sns.distplot(train['income'],bins=1000,norm_hist=False,kde=True)

In [ ]:
sns.distplot(train['age'],kde=False)

In [ ]:
sns.boxplot(x=train['experience'],y=train['age'])

In [ ]:
sns.barplot(x='experience',y='age',data=train)

In [ ]:
sns.barplot(x='experience',y='income',hue='married',data=train)

### Looks like married people are paid less as they gather experience 

### TAKE THAT SOCIETY AND SOCIAL NORMS!

In [ ]:
train.columns

In [ ]:
sns.barplot(x='house_ownership',y='income',data=train)

In [ ]:
train.groupby('experience')['age'].median()

In [ ]:
train.groupby('experience')['income'].median()

### Now, let's be honest with ourselves 

### This looks very beautiful but not a lot can actually be inferred from it, so let's jump right into feature engineering 

## Feature Engineering <a id='feature_engineering'></a>

### Time to get our hands dirty and play with the data 

### First things first let's check for duplicate entries

In [ ]:
df = train[train.duplicated()]
len(df)

### Wow, that's a lot 

### Drop duplicates to get a better model 

In [ ]:
train.drop_duplicates(inplace=True)

In [ ]:
len(train)

## Creating new column <a id='profession_state'></a>
### This part is important but it will be explained later 

In [ ]:
def profession_state(x):
    prof = x[0]
    state = x[1]
    
    new = prof + '_' + state
    return new 
        
train['profession_state']=train[['profession','state']].apply(profession_state,axis=1)
test['profession_state']=test[['profession','state']].apply(profession_state,axis=1)

In [ ]:
dummie_columns = ['married','house_ownership','car_ownership']
target_columns = ['profession','city','state','profession_state']

### Dividing the categorical columns into Dummy and Target Encoded columns

### Converting Columns with few values into dummies and others into target encoding cause of curse of dimensionality 

In [ ]:
train = pd.get_dummies(train, columns=dummie_columns, drop_first=True)
test = pd.get_dummies(test, columns=dummie_columns,drop_first=True)

In [ ]:
loo = LeaveOneOutEncoder()
train[target_columns] = loo.fit_transform(train[target_columns],train['risk_flag'])
test[target_columns] = loo.transform(test[target_columns])

## VIF(Variance Inflation Factor) <a id='vif'></a>
### VIF factor an important statistical concept playing a vital role in this question

In [ ]:
X=train.drop(columns=['risk_flag'])
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

### Important thing to note here is we remove profession and state because of their evident high vif factor and inplace introduce a new column of [profession_state](#profession_state)

### Binning age to age_bins for a better model and robust model 

In [ ]:
labels=[1,2,3,4,5]
train['age_bin'] = pd.cut(train['age'], bins=5, labels=labels).astype('uint8')
test['age_bin'] = pd.cut(test['age'], bins=5, labels=labels).astype('uint8')

### Dropping 'age','profession' and 'state' based on their high [VIF factor](#vif) and to remove multi-colinearity effect

In [ ]:
train.drop(columns=['age','profession','state'],inplace=True)
test.drop(columns=['age','profession','state'],inplace=True)

### Let's just check our VIF factor again, cause there is nothing like being too careful :-)

In [ ]:
X=train.drop(columns=['risk_flag'])
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

### I am kind of satisfied with it cause any changes just lowers the roc-auc score of the model. 

## Data Modelling <a id='data_modelling'></a>

### Time to be setup your favourite movie cause it's modelling time 

### I tried it with Boosting approach, in particular LGBMClassifier, XGBClassifier and CatBoost

### Here, LGBMClassifier fetched the best result so I am going forward with it for now 

### For feature tunning i have performed Optuna Optimization the code for the same is provided below

In [ ]:
features = ['income', 'experience', 'city', 'current_job_years',
       'current_house_years', 'profession_state','married_single',
        'house_ownership_owned', 'house_ownership_rented','car_ownership_yes', 'age_bin']

In [ ]:
data=train.drop(columns=['risk_flag'])
target=train['risk_flag']

## Optuna <a id='optuna'></a>

### The functioning of optuna can be understood through it's [documentation](https://github.com/optuna/optuna)

In [ ]:
# def objective(trial , data = data , target = target):
#     train_x , test_x , train_y , test_y = train_test_split(data , target , \
#             test_size = 0.28059109276941666 , random_state = 42)
    
#     params = {
#         'reg_alpha' : trial.suggest_loguniform('reg_alpha' , 1e-5 , 10),
#         'reg_lambda' : trial.suggest_loguniform('reg_lambda' , 1e-5 , 10),
#         'num_leaves' : trial.suggest_int('num_leaves' , 11 , 300),
#         'learning_rate' : trial.suggest_uniform('learning_rate' , 0 , 0.1),
#         'max_depth' : trial.suggest_int('max_depth' , 1 , 20),
#         'n_estimators' : trial.suggest_int('n_estimators' , 1 , 9999),
#         'min_child_samples' : trial.suggest_int('min_child_samples' , 1 , 100),
#         'min_child_weight' : trial.suggest_loguniform('min_child_weight' , 1e-5 , 1),
#         'subsample' : trial.suggest_uniform('subsample' , 0 , 1.0),
#         'colsample_bytree' : trial.suggest_loguniform('colsample_bytree' , 1e-5 , 1),
#         'random_state' : trial.suggest_categorical('random_state' , [0,42,2021,555]),
#         'metric' : 'auc',
#         'device_type' : 'gpu',
#     }
#     model = lightgbm.LGBMClassifier(**params)
#     model.fit(train_x , train_y , eval_set = [(test_x , test_y)] , early_stopping_rounds = 200 , \
#              verbose = False)
#     preds = model.predict_proba(test_x)[:,1]
#     auc = roc_auc_score(test_y , preds)
#     return auc

In [ ]:
# study = optuna.create_study(direction = 'maximize' , study_name = 'lgbm')
# study.optimize(objective , n_trials = 50)
# print('numbers of the finished trials:' , len(study.trials))
# print('the best params:' , study.best_trial.params)
# print('the best value:' , study.best_value)

### Using [Optuna](#optuna) result from above, we can model our lgbm model 

### The block given below looks terrifying but let me just break it down for you, 

### First we are taking stratified k-fold to divide the entire data into small packets of samples which will later be taken into train and test set. Since, the data is highly skewed and Binary classification problem stratified k-fold is adopted to balance out the majority and minority classes in all the samples. 

### Then, data is split and model is prepared. 

### Here LGBM model is used for training. 

### The parameters for LGBM can be seen from it's [documentation](https://lightgbm.readthedocs.io/en/latest/Parameters.html)

In [ ]:
preds=np.zeros(test.shape[0])
oof_predictions=np.zeros(len(data))
kf=StratifiedKFold(n_splits=20,random_state=42,shuffle=True)
roc=[]
n=0
for trn_idx,val_idx in kf.split(data,target):
    train_x = data.iloc[trn_idx]
    train_y = target.iloc[trn_idx]
    val_x = data.iloc[val_idx]
    val_y = target.iloc[val_idx]
    
    lgb_model = LGBMClassifier(
        random_state=42,
        cat_l2=25.999876242730252,
        cat_smooth=89.2699690675538,
        colsample_bytree=0.37861069156854954,
        early_stopping_round=200,
        learning_rate=0.015559547182527743,
        max_bin=788,
        max_depth=16,
        metric="auc",
        min_data_per_group=177,
        n_estimators=9482,
        n_jobs=-1,
        num_leaves=244,
        reg_alpha=1.8401832133621817e-05,
        reg_lambda=0.11900311978519747,
        subsample=0.9753842843857927,
        subsample_freq=1,
        verbose=-1,
    )
    lgb_model.fit(train_x,train_y,eval_set=[(val_x,val_y)],early_stopping_rounds=100,verbose=False)
    preds+=lgb_model.predict_proba(test[features])[:,1]/kf.n_splits
    oof_predictions += lgb_model.predict_proba(data[features])[:,1]/kf.n_splits
    roc.append(roc_auc_score( val_y , lgb_model.predict_proba(val_x)[:,1]))
    print(f"ROC {n+1}: {roc[n]}")
    n=n+1

## Output <a id='output'></a>
### The only thing left now is to get the test predictions and upload the csv file for submission

In [ ]:
np.mean(roc)

In [ ]:
sub['risk_flag'] = preds
sub.to_csv('lgb_1.csv' , index = False)

In [ ]:
preds_new = [1 if x>0.5 else 0 for x in preds]

In [ ]:
sub['risk_flag']=preds_new
sub.to_csv('lgb_2.csv',index=False)